In [1]:
import os, shutil

In [2]:
filelist = [
# '120523 (1)/+HGs/MTXL0.32hydrogelafter30min001.nd2',
# '102523/55min.nd2',
# '51524/ 45min.nd2',
# '32024/40min.nd2',
'012324/1-3AMT+ASE1+kinesinafter15min(3minvid).nd2',
 '42924/@20x MT only ASE1 + 0.2uMK after 35min (5min vid)002.nd2',
'021224/+ASE1/ASE1+kinesin20minsafter(5minvid)002.nd2'
]


In [3]:
dir_path = '../../../../../Volumes/Hack2024D'

for i in range(len(filelist)):
    filelist[i] = os.path.join(dir_path, filelist[i])
    # print(filelist[i])

filelist

['../../../../../Volumes/Hack2024D/012324/1-3AMT+ASE1+kinesinafter15min(3minvid).nd2',
 '../../../../../Volumes/Hack2024D/42924/@20x MT only ASE1 + 0.2uMK after 35min (5min vid)002.nd2',
 '../../../../../Volumes/Hack2024D/021224/+ASE1/ASE1+kinesin20minsafter(5minvid)002.nd2']

In [4]:
def cp_parents(volume, target_subdir, files):
    target_dir = os.path.join(volume, target_subdir)
    
    # Normalize volume path to ensure it ends with a separator
    volume = os.path.normpath(volume) + os.sep

    for file in files:
        # Ensure the file path is relative to the volume root
        if not file.startswith(volume):
            raise ValueError(f"File {file} is not within the volume {volume}")
        
        # Remove the volume part from the file path
        relative_path = os.path.relpath(file, volume)
        
        # Construct the target file path
        dest = os.path.join(target_dir, relative_path)

        # Ensure the directory exists
        dest_dir = os.path.dirname(dest)
        if not os.path.exists(dest_dir):
            print("Creating", dest_dir)
            os.makedirs(dest_dir)
        
        # Copy the file
        print("Copying %s to %s" % (file, dest))
        shutil.copy(file, dest)

In [5]:
target_dir = 'To Be Tested'
cp_parents(dir_path, target_dir, filelist)

Copying ../../../../../Volumes/Hack2024D/012324/1-3AMT+ASE1+kinesinafter15min(3minvid).nd2 to ../../../../../Volumes/Hack2024D/To Be Tested/012324/1-3AMT+ASE1+kinesinafter15min(3minvid).nd2
Copying ../../../../../Volumes/Hack2024D/42924/@20x MT only ASE1 + 0.2uMK after 35min (5min vid)002.nd2 to ../../../../../Volumes/Hack2024D/To Be Tested/42924/@20x MT only ASE1 + 0.2uMK after 35min (5min vid)002.nd2
Copying ../../../../../Volumes/Hack2024D/021224/+ASE1/ASE1+kinesin20minsafter(5minvid)002.nd2 to ../../../../../Volumes/Hack2024D/To Be Tested/021224/+ASE1/ASE1+kinesin20minsafter(5minvid)002.nd2
